In [ ]:
%pip install git+https://github.com/jiaruisong/chinese_province_city_area_mapper.git

In [ ]:
court_names = [
    "江苏省高级人民法院",
    "株洲市芦淞区人民法院",
    "湖南省高级人民法院",
    "云南省曲靖市中级人民法院",
    "湖南省长沙市中级人民法院",
    "湖南省长沙市中级人民法院",
    "上海市第二中级人民法院",
    "南宁市江南区人民法院",
    "不存在县人民法院",
    "深圳市龙岗区人民法院"
]
import cpca
df = cpca.transform(court_names)
print(df)